In [1]:
API_KEY = '11fa1d4e951478e40df536c0616abbbe'

In [3]:
import requests
import json
# from dotenv import load_dotenv

In [16]:
header = {
    "X-ELS-APIKey": API_KEY,
}

query_dict = {
    "authlast": None, 
    "authfirst": None, 
    "affil": None
}



lastname = "Werker"
firstname = "Janet"
query = f"AUTHLASTNAME%28{lastname}%29%20AND%20AUTHFIRST%28{firstname}%29"
# query = f"authlastname({lastname}) and authfirst({firstname})"

params = {
    "query":query
}

api_url = "http://api.elsevier.com/content/search/author?"

In [17]:
response = requests.get(url=api_url, params=params, headers=header)

In [18]:
print(response.text)

{"service-error":{"status":{"statusCode":"INVALID_INPUT","statusText":"Error translating query"}}}
